# Reed-Solomon Codes
Authors: Alex Matsoukas, Dakota Chang, Lila Smith

Date last edited: 05 May 2024

Description: Final project for `The Theory and Application of Algebraic Structures` at Olin College of Engineering, Spring 2024. This notebook contains an explanation of Reed-Solomon Codes and includes a basic implementation using the Python library [reedsolo](https://github.com/tomerfiliba-org/reedsolomon).

---
Reed-Solomon codes are a subset of BCH codes, introduced in 1960 in a paper by Reed and Solomon. Their resistance to burst errors has led to their use in both CDs and deep space communications where scratches or cosmic radiation could interrupt consecutive bits.

Reed-Solomon (RS) Codes are a type of cyclic code.
A *cyclic code* is a code with the following properties:

Given codewords $c_1$ and $c_2$ inside a cyclic code $C$:
- Any linear combination of $c_1$ and $c_2$ is in $C$
- Any shift of the symbols $c_1$ is in $C$ (ex: 010110 -> 101100)

If we encode any codeword $c = c_0c_1c_2c_3...c_{n-1}$ in the code $C$ as a polynomial $c(x) = c_0 + c_1x + c_2x^2 + c_3x^3 + ... + c_{n-1}x^{n-1}$, then we can treat cyclic codes as ideals in the ring $GF(q)[x] / x^n - 1$. For Reed-Solomon codes, groups of bits, called digits or symbols, are encoded rather than single bits.  

What does this mean?
- First, we can define a *field* as a set with a two-group structure. All field elements form an additive commutative group, and all non-zero field elements form a multiplicative commutative group.
- A field of finite order is called a *Galois Field*.
- $GF(q)$ is a Galois Field of order $q$, where $q$ is always a power of a prime.
- $GF(q)[x]$ is the ring of polynomials with coefficients from $GF(q)$.
- $GF(q)[x] / x^n - 1$ means that the polynomials are modded by $x^n - 1$. In other words, $x^n - 1$ wraps around to 0. In math terms, $x^n - 1 = 0$, which means that $x^n = 1$. This implies that the highest order possible for a polynomial in $GF(q)[x] / x^n - 1$ is $n - 1$.

- An *ideal* is a subring of a ring with the special property that any operation between an element in the ring and an element in the ideal results in an element inside the ideal.

With these definitions, we can define cyclic codes in a new way:

Let $c(x) = c_0 + c_1x + c_2x^2 + ... + c_{n-1}x^{n-1}$ be a member of $GF(q)[x] / x^n - 1$. Multiplying $c(x)$ by $x$ gives:

$xc(x) = c_0x + c_1x^2 + c_2x^3 + ... + c_{n-2}x^{n-1} + c_{n-1}x^n$. But since we are modding by $x^n - 1$, this is actually:

$xc(x) = c_{n-1} + c_0x + c_1x^2 + c_2x^3 + ... + c_{n-2}x^{n-1}$

In codeword form, $c(x)$ would be $c_0c_1c_2...c_{n-1}$ and $xc(x)$ would be $c_{n-1}c_0c_1c_2...c_{n-2}$.

This means that within the ring $GF(q)[x] / x^n - 1$, multiplying by $x$ is a shift of the polynomial (or codeword) to the right, and multiplying by subsequent powers of $x$ would be additional shifts. Furthermore, addition of polynomials in the ring is the same as taking linear combinations of codewords. So, an ideal in the ring $GF(q)[x] / x^n - 1$ would be one in which all shifts of a polynomial in the ideal stay in the ideal and all linear combinations of a polynomial in the ideal with another polynomial stay in the ideal - this is the definition of a cyclic code.

So, making an RS Code is a problem of finding an ideal in $GF(q)[x] / x^n - 1$. To find them, we need to be able to factor $x^n - 1$ because generators of ideals will divide $x^n - 1$. 

In any finite field $GF(q)$, there exists a *primitive element*, $\alpha$, such that the order of $\alpha$ is $q-1$; furthermore, all powers of $\alpha$ are unique, which means that every non-zero element in $GF(q)$ can be generated by $\alpha$. Additionally, if we pick our $n$ to be $q-1$, then each of the powers of $\alpha$ are now zeros of $x^n - 1 = x^{q-1} - 1$, by definition of order. RS Codes capable of correcting $t$ errors are created by taking $2t$ consecutive powers of $\alpha$. 

As you can see, with RS Codes, the length of the codeword must be tied to the size of the field.

## Key Functions Utilized in this Notebook: A Brief Overview

### `rs_generator_poly(nsym, fcr=0, generator=2)`

This function generates an irreducible generator polynomial over a finite field. In the context of fields and rings:

- **Field**: A field is a set equipped with two operations, usually addition and multiplication, where addition and multiplication are commutative, associative, and distributive over each other, and every nonzero element has a multiplicative inverse. In Reed-Solomon coding, we work with finite fields, denoted as GF(q), where "q" is a power of a prime.
- **Irreducible Polynomial**: An irreducible polynomial over a field is a polynomial that cannot be factored into the product of two non-constant polynomials over the same field. Irreducible polynomials are fundamental in creating finite fields and constructing algebraic codes like Reed-Solomon codes.

### `rs_generator_poly_all(max_nsym, fcr=0, generator=2)`

This function generates all irreducible generator polynomials up to a specified maximum number of symbols (nsym) over a finite field. It iterates through each possible number of symbols and constructs the corresponding irreducible generator polynomial using the `rs_generator_poly` function.

- **Finite Field**: A finite field, denoted as GF(q), is a field with a finite number of elements. It is constructed using an irreducible polynomial. In the context of Reed-Solomon coding, the finite field is used for arithmetic operations on symbols.
- **Generator Polynomial**: In Reed-Solomon coding, the generator polynomial defines the structure of codewords. It is constructed using irreducible polynomials over the finite field.

### `init_tables(prim=0x11d, generator=2, c_exp=8)`

This function initializes logarithm and anti-logarithm tables for efficient arithmetic operations within the finite field.

- **Logarithm and Anti-logarithm Tables**: Given any base or generator $b$, the expression $ b^{(\text{log}_b(x), \text{log}_b(y))} = x \times y$ holds true. This means that exponentiating the logarithms of two elements in a finite field with the same base results in the product of those elements. By precomputing logarithm and anti-logarithm tables using a chosen base or generator, efficient multiplication and division operations can be achieved through table lookups instead of costly exponentiation calculations.

In [1]:
# my laptop has weird things with paths, ignore this chunk
import sys
sys.path.append("/opt/homebrew/lib/python3.11/site-packages")

from tabulate import tabulate
import reedsolo as rs
import random
import string

In [2]:
temp_c_exp = 3
fast_primes = rs.find_prime_polys(c_exp=temp_c_exp, fast_primes=True, single=True)
rs_init_codes = rs.init_tables(c_exp=temp_c_exp, prim=fast_primes)

# Splitting byte arrays and joining the items with comma
bytearray1_items = ','.join([str(byte) for byte in rs_init_codes[0]])
bytearray2_items = ','.join([str(byte) for byte in rs_init_codes[1]])

# Convert items to polynomial representation
bytearray1_items_poly = ""
bytearray2_items_poly  = ""

for i in range(len(rs_init_codes[0])):
    bytearray1_items_poly += (str(rs_init_codes[0][i]) + "x^" + str(i) + " + ")

bytearray1_items_poly = bytearray1_items_poly[:-3]
    
for i in range(len(rs_init_codes[1])):
    bytearray2_items_poly += (str(rs_init_codes[1][i]) + "x^" + str(i) + " + ")

bytearray2_items_poly = bytearray2_items_poly[:-3]

# Creating tables
table1 = [["Byte Array 1", bytearray1_items, bytearray1_items_poly]]
table2 = [["Byte Array 2", bytearray2_items, bytearray2_items_poly]]

# Printing tables
print(tabulate(table1, headers=["Item", "Representation", "Polynomial Representation"]))
print("\n")
print(tabulate(table2, headers=["Item", "Representation", "Polynomial Representation"]))

Item          Representation    Polynomial Representation
------------  ----------------  -----------------------------------------------------
Byte Array 1  0,0,1,3,2,6,4,5   0x^0 + 0x^1 + 1x^2 + 3x^3 + 2x^4 + 6x^5 + 4x^6 + 5x^7


Item          Representation               Polynomial Representation
------------  ---------------------------  ---------------------------------------------------------------------------------------------------
Byte Array 2  1,2,4,3,6,7,5,1,2,4,3,6,7,5  1x^0 + 2x^1 + 4x^2 + 3x^3 + 6x^4 + 7x^5 + 5x^6 + 1x^7 + 2x^8 + 4x^9 + 3x^10 + 6x^11 + 7x^12 + 5x^13


### Prime Polynomials 

Because we are using a binary code (not, for instance, ternary), the order of our field (number of elements) will be a power of 2, based on the length of our message. For instance, if we want to have 16 bits of information, our field would be $GF(2^{4})$. In this field, we need to find an irreducible polynomial, aka a polynomial that cannot be factored in this field.

It is also worth noting that our polynomial coefficients will be from {0,1}, aka $GF(2)$, because it is a binary code. As such, we can represent the polynomial $g(x) = x^2 + 1$ as 0101. 


The fast primes algorithm will find fewer prime polynomials but will do so faster. Since we only *need* one prime polynomial for our lookup table, we can use this option. Without this option, the algorithm will check every possible polynomial (that isn't divisible by 2). For most size common sized fields, lookup tables exist that give these polynomials.


In [33]:
prim = rs.find_prime_polys(c_exp=4, fast_primes=False, single=True)
string_repr = str(bin(prim))[2:]
text_repr = [(char=="1") * f"x^{len(string_repr) - 1 - i}" for i, char in enumerate(string_repr)]
poly = " + ".join(filter(None, text_repr))
print(f"An irreducible polynomial in GF(2^4): {prim:b}, aka {poly}.")


rs.init_tables(c_exp=4, prim=prim)

An irreducible polynomial in GF(2^4): 10011, aka x^4 + x^1 + x^0.


[bytearray(b'\x00\x00\x01\x04\x02\x08\x05\n\x03\x0e\t\x07\x06\r\x0b\x0c'),
 bytearray(b'\x01\x02\x04\x08\x03\x06\x0c\x0b\x05\n\x07\x0e\x0f\r\t\x01\x02\x04\x08\x03\x06\x0c\x0b\x05\n\x07\x0e\x0f\r\t'),
 15]

In [42]:
n = 255  # length of total message+ecc
nsym = 12  # length of ecc

letters = string.ascii_lowercase  # lowercase letters
mes = ''.join(random.choice(letters) for _ in range(n - nsym))

In [43]:
mes

'mwxwtcudguivgbrsjrmoqpdgoedaeomrsocojdnxnpmuiifdxwecdgaljiukxtwjasvtahhqeutlgokuedmmqejqpmwzuoblrukgutrmsqxzihzeghayawjnympxgtvxecgtqapzymhmosdypajbguuuyjnbhozzgkvgaettqifbxykoykrtwesiqfxwdzrowjjarsupofkowlyiykzjopcmoynxdridgtjiupvaedhbmukglfg'

In [45]:
# Generator polynomial
gen = rs.rs_generator_poly_all(n)
print(gen)

{0: array('i', [1]), 1: array('i', [1, 1]), 2: array('i', [1, 3, 2]), 3: array('i', [1, 7, 14, 8]), 4: array('i', [1, 15, 54, 120, 64]), 5: array('i', [1, 31, 198, 792, 1984, 1024]), 6: array('i', [1, 63, 806, 2993, 1462, 3671, 840]), 7: array('i', [1, 127, 3302, 221, 1768, 1073, 3609, 133]), 8: array('i', [1, 255, 989, 1483, 2334, 2718, 3024, 3484, 804]), 9: array('i', [1, 511, 3722, 510, 3147, 1628, 4074, 146, 1159, 3938]), 10: array('i', [1, 1023, 2413, 1379, 2476, 3888, 2571, 3011, 1733, 969, 3496]), 11: array('i', [1, 2047, 664, 3756, 555, 2895, 3590, 943, 4066, 93, 3288, 3168]), 12: array('i', [1, 4095, 1438, 3615, 1565, 3172, 1918, 2797, 1545, 3183, 3014, 2058, 2623]), 13: array('i', [1, 3990, 2438, 2038, 321, 2558, 717, 179, 1327, 3777, 3462, 3465, 2993, 2934]), 14: array('i', [1, 3908, 2447, 2497, 2930, 3474, 2392, 2302, 2854, 727, 1583, 426, 828, 380, 1145]), 15: array('i', [1, 3808, 2063, 2370, 2943, 1008, 3534, 3803, 3439, 2170, 669, 2801, 2272, 1984, 2215, 3784]), 16: arra

In [46]:
mesecc = rs.rs_encode_msg(mes, nsym, gen=gen[nsym])

### Add errors

In [47]:
# Generating a list of 6 random indexes within the range of the list 'mesecc'.

random_list_indexes = [random.randint(0,len(mesecc)) for _ in range(6)]

print("These parts of the message, ", end="")

for i in random_list_indexes:
    print("mesecc[" + str(i) + "]: " + str(mesecc[i]), end=", ")
    mesecc[i] = 0

print("are all converted into 0 to replicate a series of errors.", end="")

These parts of the message, mesecc[153]: 106, mesecc[225]: 116, mesecc[206]: 121, mesecc[202]: 107, mesecc[17]: 114, mesecc[193]: 106, are all converted into 0 to replicate a series of errors.

Correcting errors in the message using a function `rs.rs_correct_msg()`, with parameters `mesecc` and `nsym`, then checking the corrected message.

In [51]:
rmes, recc, errata_pos = rs.rs_correct_msg(mesecc, nsym)
rs.rs_check(rmes + recc, nsym)

True

In [52]:
print('Number of detected errors and erasures: %i, their positions: %s' % (len(errata_pos), list(errata_pos)))

Number of detected errors and erasures: 6, their positions: [225, 206, 202, 193, 153, 17]
